In [1]:
# -*- encoding: utf-8 -*-
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [2]:
# 适用多项式朴素贝叶斯分类器
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
# KNN分类器
from sklearn.neighbors import KNeighborsClassifier  
# SVM支持向量机
from sklearn.svm import LinearSVC
# kmeans 聚类
from sklearn.cluster import KMeans 
# 随机森林
from sklearn.ensemble import RandomForestClassifier
# 随机梯度下降（逻辑斯蒂回归）
from sklearn import linear_model
import numpy as np

In [3]:
# 可视化
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
#模型保存与载入
from sklearn.externals import joblib
# # 注释语句用来存储你的模型
# joblib.dump(km,  'doc_cluster.pkl')
# km = joblib.load('doc_cluster.pkl')

In [5]:
# 划分数据集
def splitData(label,text):

    line_count = len(label)
    rarray=np.random.random(size=line_count)
    train_set = []
    train_tag = []
    test_set = []
    test_tag = []
    for i in range(line_count):
        if rarray[i]<0.8:
            train_set.append(text[i,:])
            train_tag.append(label[i])
        else:
            test_set.append(text[i,:])
            test_tag.append(label[i])
    return (train_set,train_tag,test_set,test_tag)

In [6]:
# 打印分类结果信息
def calculate_result(actual,pred):  
    print '***predict info***'  
    print 'precision:{0:.3f}'.format(metrics.precision_score(actual,pred,average='macro'))  
    print 'recall:{0:0.3f}'.format(metrics.recall_score(actual,pred,average='macro'))
    print 'f1-score:{0:.3f}'.format(metrics.f1_score(actual,pred,average='macro'))

In [7]:
# 分类器
def train_clf(clf,clf_name,train_set,train_target,test_set,test_target):
    print u"*** %s 分类器 ***" % (clf_name)
    # 训练模型
    clf.fit(train_set,train_target)
    # 预测
    pred = clf.predict(test_set)
    # 评估模型
    metrics.f1_score(test_target, pred, average='macro')
    calculate_result(test_target,pred)

In [8]:
# 监督学习分类
def supervisedModel(train_set,train_tag,test_set,test_tag):

#     clf_nb = MultinomialNB(alpha=.01)
    clf_svm = LinearSVC(random_state=0)
    clf_knn = KNeighborsClassifier()
#     clf_rf = RandomForestClassifier(n_estimators=100,max_depth=6, random_state=0)
    clf_sgd = linear_model.SGDClassifier(loss="log", penalty="l2")
    
#     train_clf(clf_nb,"MultinomialNB",train_set,train_tag,test_set,test_tag)
    train_clf(clf_svm,"LinearSVC",train_set,train_tag,test_set,test_tag)
    train_clf(clf_knn,"KNeighbors",train_set,train_tag,test_set,test_tag)
    train_clf(clf_sgd,"SGDClassifier",train_set,train_tag,test_set,test_tag)


In [9]:
# 无监督学习模型分类
def unSupervisedModel(train_set,train_tag,test_set,test_tag):
    clf = KMeans(n_clusters=10) 
    clf.fit(train_set)
    
    pred = clf.predict(test_set)
    # 评估模型
    metrics.f1_score(test_tag, pred, average='macro')
    calculate_result(test_tag,pred)

In [10]:
# 使用无监督学习对原始数据重新计算类别
def trainSupervisedModel(dataSet,cluster_num):
    clf = KMeans(n_clusters=cluster_num) 
    clf.fit(dataSet)
    clusterPlot(dataSet,clf.labels_)    

In [11]:
# 可视化
def clusterPlot(data,tag):
    # PCA降维
    reduced = TruncatedSVD(n_components=64).fit_transform(data)
    # TSNE降维
    embedded = TSNE(n_components=2, perplexity=30).fit_transform(reduced)
    #TSNE可视化
    fig = plt.figure(figsize=(15, 5))
    plt.subplot(131)
    plt.title('Yahoo news dataset  with TSNE')
    plt.scatter(embedded[:, 0], embedded[:, 1],c=tag, marker="x")

In [12]:
def trainModel(dataSet,target):
    # 划分数据
    train_set,train_tag,test_set,test_tag = splitData(target,dataSet)
    
#     print "train_set lenth = ",len(train_set)
#     print "test_set  lenth = ",len(test_set)
    # 监督学习
    supervisedModel(train_set,train_tag,test_set,test_tag)
    
    # 无监督学习预测
#     unSupervisedModel(train_set,train_tag,test_set,test_tag)
    
    # 使用k-means对原数据重新分类
#     trainSupervisedModel(test_set,20)    

In [13]:
def main():
    
    lsiData = "./data/textData_LSI.npy"
    ldaData = "./data/textData_LDA.npy"
    targetData = "./data/target.npy"
    
    # 读入数据 原数据矩阵需要做转置
 
    lsiDataSet = np.load(lsiData).T
    
    ldaDataSet = np.load(ldaData).T
    
    # 读入标签
    target = np.load(targetData)
    
    # 训练数据
    print "LSI data training"
    trainModel(lsiDataSet,target)
    
#     print "LDA data training"
#     trainModel(ldaDataSet,target)   
    

In [14]:
if __name__ == '__main__':
    main()